In [16]:
#import python/ml modules
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np

In [17]:
#loading the data

data = pd.read_csv('heart_failure.csv')
#print columns and their types
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB
None


In [18]:
#print out the distribution of the death_event column
print(Counter(data['DEATH_EVENT']))
#extract features and labels
x = data[['age','anaemia','creatinine_phosphokinase','diabetes','ejection_fraction','high_blood_pressure','platelets','serum_creatinine','serum_sodium','sex','smoking','time']]
y = data['DEATH_EVENT']

Counter({0: 203, 1: 96})


In [19]:
#data preprocessing

#convert categorical features in x to one-hot encoding vectors
x = pd.get_dummies(x)
#split data into training and testing categories
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [20]:
#scale the numerical features in the dataset
ct = ColumnTransformer([("numeric", StandardScaler(), ['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time'])])
#train the scaler on the training data
x_train = ct.fit_transform(x_train)
#scale the test data instance
x_test = ct.transform(x_test)

In [21]:
#encode training and test labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
#transform the encoded training and test labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [22]:
#designing the model

model = Sequential()
model.add(InputLayer(input_shape = (x_train.shape[1], )))
model.add(Dense(12, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [23]:
#train and evaluate the model

model.fit(x_train, y_train, batch_size = 16, epochs = 100)
loss, acc = model.evaluate(x_test, y_test, verbose = 0)
print('Test loss:', loss)
print('Test accuracy:', acc)

Epoch 1/100
15/15 [==============================] - 0s 619us/step - loss: 0.7909 - accuracy: 0.4603
Epoch 2/100
15/15 [==============================] - 0s 488us/step - loss: 0.7481 - accuracy: 0.5188
Epoch 3/100
15/15 [==============================] - 0s 443us/step - loss: 0.7142 - accuracy: 0.5523
Epoch 4/100
15/15 [==============================] - 0s 390us/step - loss: 0.6826 - accuracy: 0.6276
Epoch 5/100
15/15 [==============================] - 0s 362us/step - loss: 0.6546 - accuracy: 0.6736
Epoch 6/100
15/15 [==============================] - 0s 595us/step - loss: 0.6287 - accuracy: 0.7322
Epoch 7/100
15/15 [==============================] - 0s 528us/step - loss: 0.6070 - accuracy: 0.7573
Epoch 8/100
15/15 [==============================] - 0s 483us/step - loss: 0.5847 - accuracy: 0.7741
Epoch 9/100
15/15 [==============================] - 0s 362us/step - loss: 0.5657 - accuracy: 0.7741
Epoch 10/100
15/15 [==============================] - 0s 344us/step - loss: 0.5474 - accura

In [24]:
#generating a classification report
y_estimate = model.predict(x_test)
y_estimate = np.argmax(y_estimate, axis = 1)
y_true = np.argmax(y_test, axis = 1)
print(classification_report(y_true, y_estimate))

2/2 [==============================] - 0s 911us/step
              precision    recall  f1-score   support

           0       0.76      0.95      0.84        37
           1       0.86      0.52      0.65        23

    accuracy                           0.78        60
   macro avg       0.81      0.73      0.75        60
weighted avg       0.80      0.78      0.77        60
